<a href="https://colab.research.google.com/github/siva-sankar-a/eva_final_project/blob/master/Dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import glob
import cv2
import os
import shutil
import numpy as np
import pandas as pd

In [0]:
root_dir = '/content/drive/My Drive/eva_final_data'

bg_folder_raw = os.path.join(root_dir, 'bg_raw')
bg_folder_selected = os.path.join(root_dir, 'bg_selected')
mask_folder_selected = os.path.join(root_dir, 'mask_selected')
fg_folder_selected = os.path.join(root_dir, 'fg_selected')
bg_folder_cropped = os.path.join(root_dir, 'bg_cropped')
fg_folder_cropped = os.path.join(root_dir, 'fg_cropped')
mask_folder_cropped = os.path.join(root_dir, 'mask_cropped')

mask_folder = os.path.join(root_dir, 'mask')
fg_bg_folder = os.path.join(root_dir, 'fg_bg')

dataset_dirs = [bg_folder_raw, bg_folder_selected, mask_folder_selected, fg_folder_selected, bg_folder_cropped,
               fg_folder_cropped, mask_folder_cropped, mask_folder, fg_bg_folder]

for dataset_dir in dataset_dirs: 
    if not os.path.exists(dataset_dir):
        os.mkdir(dataset_dir)

H = 224
W = 224

_W = 80
_H = 70

### Copy selected background images to bg_selected folder

In [0]:
paths = glob.glob(f'{bg_folder_raw}/*.*')

selected_paths = []
for idx, path in enumerate(paths):
    img = cv2.imread(path)
    h, w, c = img.shape
    if h > H and w > W:
        selected_paths.append(path)

for idx, selected_path in enumerate(selected_paths):
    shutil.copy(selected_path, f'{bg_folder_selected}/img_{idx:03d}.png')

### Segregate mask and foreground images into mask_selected and fg_selected images

In [0]:
paths = glob.glob(os.path.join(root_dir, 'no_bkg_and_mask') + '/*.*')
for path in paths:
    base_name = os.path.basename(path)
    if '_m' in base_name:
        actual_basename = base_name.replace('_m', '')
        shutil.copy(path, os.path.join(mask_folder_selected, actual_basename))
    else:
        actual_basename = base_name
        shutil.copy(path, os.path.join(fg_folder_selected, actual_basename))

### Resize images to 224, 224

Since the images selected are of very high resolution and cropping them makes them loose perspective data which results in bad disparity maps, they were resized

In [0]:
paths = glob.glob(f'{bg_folder_selected}/*.*')
for idx, path in enumerate(paths):
    base_name = os.path.basename(path)
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    h, w, c = img.shape
    
    if c == 3:
        b, g, r = cv2.split(img)
        a = np.ones((h, w), dtype=b.dtype) * 255
        img = cv2.merge((b, g, r, a))
    
    dst = cv2.resize(img, dsize=(H, W), interpolation=cv2.INTER_CUBIC)
    
    path_to_cropped_img = os.path.join(fg_folder_selected, base_name)
    cv2.imwrite(f'{bg_folder_cropped}/img_{idx:03d}.png', dst)

### Finding average size of foreground images

In [37]:
paths = glob.glob(f'{fg_folder_selected}/*.*')

img_shapes = []
for idx, path in enumerate(paths):
    img = cv2.imread(path)
    img_shapes.append(img.shape)

np.stack(pd.Series(img_shapes).apply(list).values).mean(axis=0) / 10

array([73.775, 84.035,  0.3  ])

### Resize foreground images to 70 X 80 and dump it to foreground_cropped folder

In [0]:
paths = glob.glob(f'{fg_folder_selected}/*.*')
for idx, path in enumerate(paths):
    base_name = os.path.basename(path)
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    h, w, c = img.shape

    if c == 3:
        b, g, r = cv2.split(img)
        a = np.ones((h, w), dtype=b.dtype) * 255
        img = cv2.merge((b, g, r, a))
        
    dst = cv2.resize(img, dsize=(_W, _H), interpolation=cv2.INTER_CUBIC)
    
    path_to_cropped_img = os.path.join(fg_folder_cropped, base_name)
    cv2.imwrite(path_to_cropped_img, dst)

### Resize mask images to 70 X 80 and dump it to mask_cropped folder

In [0]:
paths = glob.glob(f'{mask_folder_selected}/*.*')
for idx, path in enumerate(paths):
    base_name = os.path.basename(path)
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    h, w, c = img.shape
    
    if c == 3:
        b, g, r = cv2.split(img)
        a = np.ones((h, w), dtype=b.dtype) * 255
        img = cv2.merge((b, g, r, a))

    dst = cv2.resize(img, dsize=(_W, _H), interpolation=cv2.INTER_CUBIC)
    
    path_to_cropped_img = os.path.join(mask_folder_cropped, base_name)
    cv2.imwrite(path_to_cropped_img, dst)

# Dataset Generation

In [0]:
fg_paths = glob.glob(f'{fg_folder_cropped}/*.*')
cropped_mask_paths = glob.glob(f'{mask_folder_cropped}/*.*')
bg_paths = glob.glob(f'{bg_folder_cropped}/*.*')

In [0]:
# Set compression level and number of random placements for each foreground background pair
compression_level = 75
number_of_random_placements = 20

In [0]:
fg_bg_paths = []
mask_paths = []

In [0]:
## Iterate through all backgrounds
for bg_path in bg_paths:
    bg_img = cv2.imread(bg_path, cv2.IMREAD_UNCHANGED)
    i = 0
    
    ## Iterate through all foregrounds
    for fg_path in fg_paths:
        fg_base_name = os.path.basename(fg_path)
        mask_path = os.path.join(mask_folder_cropped, fg_base_name)
        
        ## If mask is found
        if mask_path in cropped_mask_paths:
            fg_img = cv2.imread(fg_path, cv2.IMREAD_UNCHANGED)
            mask_img = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            
            ## Iterate through all random placement possibilities
            for _ in range(number_of_random_placements):
                
                ## Randomly place fg on bg
                r_min = 0
                r_max = H - _H

                c_min = 0
                c_max = W - _W

                r = np.random.randint(r_min, r_max)
                c = np.random.randint(c_min, c_max)

                random_mask_img = np.zeros_like(bg_img, dtype=bg_img.dtype)
                random_mask_img[:, :, 3].fill(255)

                random_fg_img = np.zeros_like(bg_img, dtype=bg_img.dtype)
                random_fg_img[:, :, 3].fill(255)

                random_mask_img[r: r + _H, c: c + _W] += mask_img
                random_fg_img[r: r + _H, c: c + _W] += fg_img
                
                random_mask_img = cv2.cvtColor(random_mask_img, cv2.COLOR_BGRA2GRAY)
                random_bg_img = cv2.bitwise_and(bg_img, bg_img, mask=cv2.bitwise_not(random_mask_img))
                random_bg_img += random_fg_img
                
                random_bg_img = cv2.cvtColor(random_bg_img, cv2.COLOR_BGRA2BGR)
                
                fg_bg_path = f'{fg_bg_folder}/img_{i:08d}.jpg'
                mask_path = f'{mask_folder}/img_{i:08d}.jpg'
                
                cv2.imwrite(mask_path, random_mask_img, [int(cv2.IMWRITE_JPEG_QUALITY), compression_level])
                cv2.imwrite(fg_bg_path, random_bg_img,  [int(cv2.IMWRITE_JPEG_QUALITY), compression_level])
                
                fg_bg_paths.append(fg_bg_path)
                mask_paths.append(mask_path)
                
                i += 1
                
                ## Flip the image and mask
                fg_img = cv2.flip(fg_img, 1)
                mask_img = cv2.flip(mask_img, 1)

                ## Randomly place fg on bg
                r_min = 0
                r_max = H - _H

                c_min = 0
                c_max = W - _W

                r = np.random.randint(r_min, r_max)
                c = np.random.randint(c_min, c_max)

                random_mask_img = np.zeros_like(bg_img, dtype=bg_img.dtype)
                random_mask_img[:, :, 3].fill(255)

                random_fg_img = np.zeros_like(bg_img, dtype=bg_img.dtype)
                random_fg_img[:, :, 3].fill(255)

                random_mask_img[r: r + _H, c: c + _W] += mask_img
                random_fg_img[r: r + _H, c: c + _W] += fg_img

                random_mask_img = cv2.cvtColor(random_mask_img, cv2.COLOR_BGRA2GRAY)
                random_bg_img = cv2.bitwise_and(bg_img, bg_img, mask=cv2.bitwise_not(random_mask_img))
                random_bg_img += random_fg_img

                random_bg_img = cv2.cvtColor(random_bg_img, cv2.COLOR_BGRA2BGR)
                
                fg_bg_path = f'{fg_bg_folder}/img_{i:08d}.jpg'
                mask_path = f'{mask_folder}/img_{i:08d}.jpg'
                
                cv2.imwrite(mask_path, random_mask_img, [int(cv2.IMWRITE_JPEG_QUALITY), compression_level])
                cv2.imwrite(fg_bg_path, random_bg_img,  [int(cv2.IMWRITE_JPEG_QUALITY), compression_level])
                
                fg_bg_paths.append(fg_bg_path)
                mask_paths.append(mask_path)
                
                i += 1

In [0]:
df = pd.DataFrame({'fg_bg_paths': fg_bg_paths, 'mask_paths': mask_paths})
df.to_csv('file_info.csv', index=False)